# Autograd Tutorial



## Approaches for Computing Derivatives

* **Symbolic differentiation:** automatic manipulation of mathematical expressions to get derivatives
    - Takes a math expression and returns a math expression: $f(x) = x^2 \rightarrow \frac{df(x)}{dx} = 2x$
    - Used in Mathematica, Maple, Sympy, etc.
* **Numeric differentiation:** Approximating derivatives by finite differences:
$$
\frac{\partial}{\partial x_i} f(x_1, \dots, x_N) = \lim_{h \to 0} \frac{f(x_1, \dots, x_i + h, \dots, x_N) - f(x_1, \dots, x_i - h, \dots, x_N)}{2h}
$$
* **Automatic differentiation:** Takes code that computes a function and returns code that computes the derivative of that function.
    - Reverse Mode AD: A method to get exact derivatives efficiently, by storing information as you go forward that you can reuse as you go backwards
    - "The goal isn't to obtain closed-form solutions, but to be able to wirte a program that efficiently computes the derivatives." - Lecture 6 Slides (Backpropagation)
    - **Autograd**, **Torch Autograd**

## Autograd

* [Autograd](https://github.com/HIPS/autograd) is a Python package for automatic differentiation.
* To install Autograd:
                pip install autograd
* There are a lot of great [examples](https://github.com/HIPS/autograd/tree/master/examples) provided with the source code

### What can Autograd do?

From the Autograd Github repository:

* Autograd can automatically differentiate native Python and Numpy code.
* It can handle a large subset of Python's features, including loops, conditional statements (if/else), recursion and closures
* It can also compute higher-order derivatives
* It uses reverse-mode differentiation (a.k.a. backpropagation) so it can efficiently take gradients of scalar-valued functions with respect to array-valued arguments.


## Autograd vs Tensorflow, Theano, etc.

Many deep learning packages implement automatic differentiation using small _domain-specific languages_ within Python:
    - Theano
    - Caffe
    - Vanilla Torch (as compared to Autograd for Torch)
    - Tensorflow
Most of these alternatives require you to _explicitly_ construct a computation graph; Autograd constructs a computation graph _implicitly_, by tracking the sequence of operations that have been performed during the execution of a program.

## GPU Support for Autograd?

There are a couple projects to look into if you are interested in GPU support for Autograd:

* [MinPy](https://github.com/dmlc/minpy)
* [PyTorch](http://pytorch.org/)

## Autograd Basic Usage

In [1]:
# Import thinly-wrapped numpy
import autograd.numpy as np
# Basicallly the only function you need
from autograd import grad

In [13]:
# Define a normal function, using python & numpy
def tanh(x):
    y = np.exp(-x)
    return (1.-y)/(1.+y)

grad_tanh = grad(tanh)

# Evaluate the gradient at x=10
print(grad_tanh(1.0))

# Compare to numeric gradient computed using finite differences
print((tanh(1.0001) - tanh(0.9999)) / 0.0002)

0.39322386648296376
0.39322386636453377


## Autograd vs Manual Gradients via Staged Computation

In this example, we will see how a complicated computation can be written as a composition of simpler functions, and how this provides a scalable strategy for computing gradients using the chain rule.

Say we want to write a function to compute the gradient of the *sigmoid function*:
$$
\sigma(x) = \frac{1}{1 + e^{-x}}
$$
We can write $\sigma(x)$ as a composition of several elementary functions, as $\sigma(x) = s(c(b(a(x))))$, where:

$$
a(x) = -x
$$

$$
b(a) = e^a
$$

$$
c(b) = 1 + b
$$

$$
s(c) = \frac{1}{c}
$$

Here, we have "staged" the computation such that it contains several intermediate variables, each of which are basic expressions for which we can easily compute the local gradients.

The computation graph for this expression is shown in the figure below. 

![Gradient Computation Image](images/ComputationGraph.png)

The input to this function is $x$, and the output is represented by node $s$. We wish compute the gradient of $s$ with respect to $x$, $\frac{\partial s}{\partial x}$. In order to make use of our intermediate computations, we can use the chain rule as follows:
$$
\frac{\partial s}{\partial x} = \frac{\partial s}{\partial c} \frac{\partial c}{\partial b} \frac{\partial b}{\partial a} \frac{\partial a}{\partial x}
$$

<!--
Given a vector-to-scalar function, $\mathbb{R}^D \to \mathbb{R}$, composed of a set of primitive functions
$\mathbb{R}^M \to \mathbb{R}^N$ (for various $M$, $N$), the gradient of the composition is given by the product of the gradients of the primitive functions, according to the chain rule. But the chain rule doesn’t prescribe the order in which to multiply the gradients. From the perspective of computational complexity, the order makes all the
difference.
-->

In [18]:
def sigmoid(x):
    y = 1.0 / (1.0 + np.exp(-x))
    return y

def grad_sigmoid_manual(x):
    '''
    Define the gradient of the logistic sigmoid function
    '''
    # First define the elementary functions of the sigmoid function as seen above.
    a = -x         
    b = np.exp(a)  
    c = 1 + b      
    s = 1.0 / c 
    
    # Backward pass
    dsdc = (-1.0 / (c**2))
    dsdb = dsdc * 1
    dsda = dsdb * np.exp(a)
    dsdx = dsda * (-1)
    
    return dsdx

## Manually without Autograd
print('Manual calculation of gradient:   ',grad_sigmoid_manual(2.0))

## Automatically with Autograd
grad_sigmoid_automatic = grad(sigmoid)
print('Automatic calculation of gradient:',grad_sigmoid_automatic(2.0))

Manual calculation of gradient:    0.1049935854035065
Automatic calculation of gradient: 0.1049935854035065


I know what you're thinking... Autograd is amazing!!!

## Gradients of Data Structures: flatten and unflatten
                                                                     
Autograd allows you to compute gradients for many different data structures. Autograd provides a lot of flexibility in the types of data structures you can use to store the parameters of your model. This flexibility is achieved through the flatten function, which converts any nested combination of lists, tuples, arrays, or dicts into a 1-dimensional Numpy array.                                              
                                                                                                      
The idea is that we know how to compute gradients of vectors, and we can convert many data structures into vectors (i.e. "flatten" the data structures).

In [19]:
import autograd.numpy as np
import autograd.numpy.random as npr
from autograd import grad
from autograd.misc import flatten

In [20]:
# You can flatten a list of tupples
params = [(1.0, 1.0), (2.0,2.0), (3.0,3.0,3.0)]
flat_params, unflatten_func = flatten(params)
print('Flattened: {}'.format(flat_params))
print('Unflattened: {}'.format(unflatten_func(flat_params)))

Flattened: [1. 1. 2. 2. 3. 3. 3.]
Unflattened: [(array(1.), array(1.)), (array(2.), array(2.)), (array(3.), array(3.), array(3.))]
